# Generative AI with LLM based autonomous agents augmented with structured and unstructured data

We will use Neural Search plugin in OpenSearch to implement semantic search

### Check PyTorch Version


In [ ]:
import torch
print(torch.__version__)

###  Install OpenSearch ML Python library

In [ ]:
!pip install -q opensearch-py
%pip install langchain
%pip install boto3
%pip install sqlalchemy>
%pip install sqlalchemy-redshift
%pip install redshift_connector
%pip install ipython-sql==0.4.1
%pip install langchain_experimental

### Import library



In [ ]:
import boto3
import re
import time
import sagemaker,json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

## Part 1. Prepare unstructrued data in OpenSearch

### Prepare SEC Edgar Annual Financial Filings data

Download the dataset from Kaggle. Then upload the downloaded "archive.zip" to same folder as this notebook.

https://www.kaggle.com/datasets/pranjalverma08/sec-edgar-annual-financial-filings-2021/

In [ ]:
!unzip archive.zip

In [ ]:
import os
import pandas as pd

# Specify the path to the folder containing the JSON files
folder_path = "extracted"

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        df = pd.DataFrame([pd.read_json(file_path,typ='series')]
        dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)



In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
combined_df

In [ ]:
print(combined_df.loc[0]["item_1"])

### Create an OpenSearch cluster connection.
Next, we'll use Python API to set up connection with OpenSearch Cluster.

Note: if you're using a region other than us-east-1, please update the region in the code below.

#### Get Cloud Formation stack output variables

We also need to grab some key values from the infrastructure we provisioned using CloudFormation. To do this, we will list the outputs from the stack and store this in "outputs" to be used later.

You can ignore any "PythonDeprecationWarning" warnings.

In [ ]:
import json
region = aws_region

cfn = boto3.client('cloudformation')
kms = boto3.client('secretsmanager')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "rag-with-redshift"

outputs = get_cfn_outputs(cloudformation_stack_name)
aos_host = outputs['OpenSearchDomainEndpoint']
aos_credentials = json.loads(kms.get_secret_value(SecretId=outputs['OpenSearchSecret'])['SecretString'])

outputs

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection

auth = (aos_credentials['username'], aos_credentials['password'])
aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

### Create a index in Amazon Opensearch Service 
Whereas we previously created an index with 2 fields, this time we'll define the index with 3 fields: the first field ' question_vector' holds the vector representation of the question, the second is the "question" for raw sentence and the third field is "answer" for the raw answer data.

To create the index, we first define the index in JSON, then use the aos_client connection we initiated ealier to create the index in OpenSearch.

In [ ]:
knn_index = {
    "settings": {
        "index.knn": True,
        "index.knn.space_type": "cosinesimil"
    },
    "mappings": {
        "properties": {
            "item_vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "store": True
            },
            "item_content": {
                "type": "text",
                "store": True
            },
            "item_category": {
                "type": "text",
                "store": True
            },
            "company_name": {
                "type": "text",
                "store": True
            }
        }
    }
}

Using the above index definition, we now need to create the index in Amazon OpenSearch

In [ ]:
index_name="10k_financial"
aos_client.indices.create(index=index_name,body=knn_index,ignore=400)

Let's verify the created index information

In [ ]:
aos_client.indices.get(index=index_name)

In [ ]:
from langchain.embeddings import BedrockEmbeddings


boto3_bedrock = boto3.client(service_name="bedrock-runtime", endpoint_url=f"https://bedrock-runtime.{aws_region}.amazonaws.com")
bedrock_embeddings = BedrockEmbeddings(model_id='amazon.titan-embed-text-v1',client=boto3_bedrock)
result = bedrock_embeddings.embed_query("This is a content of the document")


In [ ]:
result

###  Load the raw data into the Index
Next, let's load the financial billing data into the index we've just created.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 8000, chunk_overlap = 500)

company_list=['MICROSOFT CORP', 'SPLUNK INC', 'Dynatrace, Inc']

items=["item_1","item_1A","item_1B","item_2","item_3","item_4","item_5","item_6","item_7","item_7A","item_8","item_9","item_9A", "item_9B", "item_10", "item_11", "item_12", "item_13", "item_14", "item_15"]
for index, row in combined_df.iterrows():
    company_name=row['company']
    if company_name in company_list:
        for item in items:
            item_content = row[item]
            with open("item.txt", 'w') as file:
                file.write(item_content)
            loader = TextLoader("item.txt")
            splitted_texts = loader.load_and_split(text_splitter=text_splitter)
            for text in splitted_texts:
                embedding_result = bedrock_embeddings.embed_query(text.page_content)
                aos_client.index(index=index_name,body={ "company_name": company_name,"item_category":item, "item_content":text.page_content, "item_vector":embedding_result})

To validate the load, we'll query the number of documents number in the index. We should have 1000 hits in the index.

In [ ]:
res = aos_client.search(index=index_name, body={"query": {"match_all": {}}})
print("Records found: %d." % res['hits']['total']['value'])

In [ ]:
#aos_client.indices.delete(index=index_name)


## Part 2 - Prepare structured data  in Redshift Database

In [ ]:
redshift_serverless_credentials = json.loads(kms.get_secret_value(SecretId=outputs['RedshiftServerlessSecret'])['SecretString'])
redshift_serverless_username=redshift_serverless_credentials['username']
redshift_serverless_password=redshift_serverless_credentials['password']
redshift_serverless_endpoint =  outputs['RedshiftServerlessEndpoint']

Get Redshift Serverless workgroup and query if it is public accessible.

In [ ]:
import boto3
import time

redshift_serverless_client = boto3.client('redshift-serverless')
redshift_serverless_workgroup = outputs['RedshiftServerlessWorkroup']

get_response = redshift_serverless_client.get_workgroup(
    workgroupName=redshift_serverless_workgroup
)

print("publiclyAccessible: " + str(get_response['workgroup']['publiclyAccessible']))


If Redshift Serverless Endpoint is not public accessible, we have to change it to public accessible to connect it from this Notebook.

### Note: We have to restore the public accessible if you don't want to expose Redshift Serverless Endpont to public.

In [ ]:
if get_response['workgroup']['publiclyAccessible'] == False:
    redshift_serverless_client.update_workgroup(workgroupName=redshift_serverless_workgroup, publiclyAccessible=True)

get_response = redshift_serverless_client.get_workgroup(
    workgroupName=redshift_serverless_workgroup
)
update_status = get_response['workgroup']['status']

while update_status == 'MODIFYING':
    time.sleep(2)
    print('.', end='')
    get_response = redshift_serverless_client.get_workgroup(
    workgroupName=redshift_serverless_workgroup
    )
    update_status = get_response['workgroup']['status']

    

In [ ]:
get_response = redshift_serverless_client.get_workgroup(
    workgroupName=redshift_serverless_workgroup
)

print("publiclyAccessible: " + str(get_response['workgroup']['publiclyAccessible']))


In [ ]:
import sqlalchemy as sa
from sqlalchemy.engine.url import URL
from sqlalchemy.orm import Session
%reload_ext sql
%config SqlMagic.displaylimit = 25

connect_to_db = URL.create(
drivername='redshift+redshift_connector', # indicate redshift_connector driver and dialect will be used
host=redshift_serverless_endpoint, 
port=5439,
database='dev',
username=redshift_serverless_username,
password=redshift_serverless_password
)

%sql $connect_to_db
%sql select current_user, version();

### Stock symbol table

In [ ]:
%sql CREATE TABLE IF NOT EXISTS public.stock_symbol (stock_symbol text PRIMARY KEY, company_name text NOT NULL);


In [ ]:
%sql INSERT INTO public.stock_symbol (stock_symbol, company_name) VALUES ('AMZN','Amazon.com, Inc.')
%sql INSERT INTO public.stock_symbol (stock_symbol, company_name) VALUES ('TSLA','Tesla Inc.')
%sql INSERT INTO public.stock_symbol (stock_symbol, company_name) VALUES ('PAAS','Pan American Silver Corp.')
%sql INSERT INTO public.stock_symbol (stock_symbol, company_name) VALUES ('PAAC','Proficient Alpha Acquisition Corp.')
%sql INSERT INTO public.stock_symbol (stock_symbol, company_name) VALUES ('MSFT','Microsoft Corp.')

In [ ]:
%sql select * from public.stock_symbol

### Stock price table

Create stock price table

In [ ]:
%sql CREATE TABLE IF NOT EXISTS public.stock_price (stock_date DATE, stock_symbol text, open_price DECIMAL, high_price DECIMAL, low_price DECIMAL, close_price DECIMAL, adjusted_close_price DECIMAL, volume DECIMAL);



Populate the Redshift table 

In [ ]:
!rm MSFT.sql

with open("MSFT.csv", 'r') as file:
    with open("MSFT.sql", 'a') as sql_file:
        count = 0
        line = file.readline()
        line = file.readline()
        while line:
            line = line.rstrip()
            items = line.split(",")
            sql = """%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('{}','{}',{},{},{},{},{},{})""".format(items[0],items[1],items[2],items[3],items[4],items[5],items[6],items[7])
            sql_file.write(sql)
            sql_file.write("\n")
            line = file.readline()
            count += 1
        

In [ ]:
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-06-29','MSFT',257.5899963378906,261.9700012207031,255.75999450683594,260.260009765625,257.2237548828125,20069800)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-06-30','MSFT',257.04998779296875,259.5299987792969,252.89999389648438,256.8299865722656,253.83377075195312,31730900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-01','MSFT',256.3900146484375,259.7699890136719,254.61000061035156,259.5799865722656,256.5516357421875,22837700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-05','MSFT',256.1600036621094,262.9800109863281,254.74000549316406,262.8500061035156,259.7835388183594,22941000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-06','MSFT',263.75,267.989990234375,262.3999938964844,266.2099914550781,263.1043395996094,23824400)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-07','MSFT',265.1199951171875,269.05999755859375,265.0199890136719,268.3999938964844,265.268798828125,20859900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-08','MSFT',264.7900085449219,268.1000061035156,263.2900085449219,267.6600036621094,264.5373840332031,19658800)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-11','MSFT',265.6499938964844,266.5299987792969,262.17999267578125,264.510009765625,261.4241638183594,19455200)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-12','MSFT',265.8800048828125,265.94000244140625,252.0399932861328,253.6699981689453,250.7106475830078,35868500)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-13','MSFT',250.19000244140625,253.5500030517578,248.11000061035156,252.72000122070312,249.771728515625,29497400)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-14','MSFT',250.57000732421875,255.13999938964844,245.94000244140625,254.0800018310547,251.1158447265625,25102800)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-15','MSFT',255.72000122070312,260.3699951171875,254.77000427246094,256.7200012207031,253.72506713867188,29774100)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-18','MSFT',259.75,260.8399963378906,253.3000030517578,254.25,251.2838897705078,20975000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-19','MSFT',257.5799865722656,259.7200012207031,253.67999267578125,259.5299987792969,256.5022888183594,25012600)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-20','MSFT',259.8999938964844,264.8699951171875,258.9100036621094,262.2699890136719,259.2102966308594,22788300)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-21','MSFT',259.7900085449219,264.8900146484375,257.0299987792969,264.8399963378906,261.75030517578125,22404700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-22','MSFT',265.239990234375,265.3299865722656,259.07000732421875,260.3599853515625,257.32257080078125,21881300)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-25','MSFT',261.0,261.5,256.80999755859375,258.8299865722656,255.81044006347656,21056000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-26','MSFT',259.8599853515625,259.8800048828125,249.57000732421875,251.89999389648438,248.96128845214844,39348000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-27','MSFT',261.1600036621094,270.04998779296875,258.8500061035156,268.739990234375,265.60479736328125,45994000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-28','MSFT',269.75,277.8399963378906,267.8699951171875,276.4100036621094,273.18536376953125,33459300)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-07-29','MSFT',277.70001220703125,282.0,276.6300048828125,280.739990234375,277.4648132324219,32152800)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-01','MSFT',277.82000732421875,281.2799987792969,275.8399963378906,278.010009765625,274.7666931152344,21539600)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-02','MSFT',276.0,277.8900146484375,272.3800048828125,274.82000732421875,271.6138916015625,22754200)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-03','MSFT',276.760009765625,283.5,276.6099853515625,282.4700012207031,279.17462158203125,23518900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-04','MSFT',281.79998779296875,283.79998779296875,280.1700134277344,283.6499938964844,280.34088134765625,18098700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-05','MSFT',279.1499938964844,283.6499938964844,278.67999267578125,282.9100036621094,279.6095275878906,16774600)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-08','MSFT',284.04998779296875,285.9200134277344,279.32000732421875,280.32000732421875,277.0497131347656,18739200)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-09','MSFT',279.6400146484375,283.0799865722656,277.6099853515625,282.29998779296875,279.0066223144531,23405200)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-10','MSFT',288.1700134277344,289.80999755859375,286.94000244140625,289.1600036621094,285.78662109375,24687800)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-11','MSFT',290.8500061035156,291.2099914550781,286.510009765625,287.0199890136719,283.67156982421875,20065900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-12','MSFT',288.4800109863281,291.9100036621094,286.94000244140625,291.9100036621094,288.5044860839844,22619700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-15','MSFT',291.0,294.17999267578125,290.1099853515625,293.4700012207031,290.04632568359375,18085700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-16','MSFT',291.989990234375,294.0400085449219,290.4200134277344,292.7099914550781,289.2951354980469,18102900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-17','MSFT',289.739990234375,293.3500061035156,289.4700012207031,291.32000732421875,288.53253173828125,18253400)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-18','MSFT',290.19000244140625,291.9100036621094,289.0799865722656,290.1700134277344,287.3935546875,17186200)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-19','MSFT',288.8999938964844,289.25,285.55999755859375,286.1499938964844,283.4119873046875,20570000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-22','MSFT',282.0799865722656,282.4599914550781,277.2200012207031,277.75,275.0923767089844,25061100)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-23','MSFT',276.44000244140625,278.8599853515625,275.3999938964844,276.44000244140625,273.7948913574219,17527400)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-24','MSFT',275.4100036621094,277.2300109863281,275.1099853515625,275.7900085449219,273.1511535644531,18137000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-25','MSFT',277.3299865722656,279.0199890136719,274.5199890136719,278.8500061035156,276.181884765625,16583400)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-26','MSFT',279.0799865722656,280.3399963378906,267.9800109863281,268.0899963378906,265.52484130859375,27549300)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-29','MSFT',265.8500061035156,267.3999938964844,263.8500061035156,265.2300109863281,262.6922302246094,20338500)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-30','MSFT',266.6700134277344,267.04998779296875,260.6600036621094,262.9700012207031,260.4538269042969,22767100)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-08-31','MSFT',265.3900146484375,267.1099853515625,261.3299865722656,261.4700012207031,258.9681701660156,24791800)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-01','MSFT',258.8699951171875,260.8900146484375,255.41000366210938,260.3999938964844,257.90838623046875,23263400)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-02','MSFT',261.70001220703125,264.739990234375,254.47000122070312,256.05999755859375,253.60992431640625,22855400)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-06','MSFT',256.20001220703125,257.8299865722656,251.94000244140625,253.25,250.8268280029297,21328200)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-07','MSFT',254.6999969482422,258.8299865722656,253.22000122070312,258.0899963378906,255.62049865722656,24126700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-08','MSFT',257.510009765625,260.42999267578125,254.7899932861328,258.5199890136719,256.0463562011719,20319900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-09','MSFT',260.5,265.2300109863281,260.2900085449219,264.4599914550781,261.9295654296875,22084700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-12','MSFT',265.7799987792969,267.45001220703125,265.1600036621094,266.6499938964844,264.0986022949219,18747700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-13','MSFT',258.8399963378906,260.3999938964844,251.58999633789062,251.99000549316406,249.57887268066406,33353300)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-14','MSFT',253.52999877929688,254.22999572753906,249.86000061035156,252.22000122070312,249.80667114257812,23913000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-15','MSFT',249.77000427246094,251.39999389648438,244.02000427246094,245.3800048828125,243.03211975097656,31530900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-16','MSFT',244.25999450683594,245.3000030517578,242.05999755859375,244.74000549316406,242.3982391357422,39791800)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-19','MSFT',242.47000122070312,245.13999938964844,240.85000610351562,244.52000427246094,242.1803436279297,26826900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-20','MSFT',242.07000732421875,243.50999450683594,239.63999938964844,242.4499969482422,240.13015747070312,26660300)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-21','MSFT',244.27000427246094,247.66000366210938,238.89999389648438,238.9499969482422,236.66363525390625,28625600)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-22','MSFT',237.8699951171875,243.86000061035156,237.57000732421875,240.97999572753906,238.67420959472656,31061200)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-23','MSFT',239.5399932861328,241.1300048828125,235.1999969482422,237.9199981689453,235.64349365234375,34176000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-26','MSFT',237.0500030517578,241.4499969482422,236.89999389648438,237.4499969482422,235.17799377441406,27694200)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-27','MSFT',239.97999572753906,241.8000030517578,234.5,236.41000366210938,234.14794921875,27018700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-28','MSFT',236.80999755859375,242.3300018310547,234.72999572753906,241.07000732421875,238.76336669921875,29029700)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-29','MSFT',238.88999938964844,239.9499969482422,234.41000366210938,237.5,235.22750854492188,27484200)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-09-30','MSFT',238.2899932861328,240.5399932861328,232.72999572753906,232.89999389648438,230.6715087890625,35694800)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-03','MSFT',235.41000366210938,241.61000061035156,234.66000366210938,240.74000549316406,238.4365234375,28880400)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-04','MSFT',245.08999633789062,250.36000061035156,244.97999572753906,248.8800048828125,246.49864196777344,34888400)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-05','MSFT',245.99000549316406,250.5800018310547,244.10000610351562,249.1999969482422,246.8155517578125,20347100)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-06','MSFT',247.92999267578125,250.33999633789062,246.0800018310547,246.7899932861328,244.42861938476562,20239900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-07','MSFT',240.89999389648438,241.32000732421875,233.1699981689453,234.24000549316406,231.99871826171875,37769600)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-10','MSFT',233.0500030517578,234.55999755859375,226.72999572753906,229.25,227.05645751953125,29743600)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-11','MSFT',227.6199951171875,229.05999755859375,224.11000061035156,225.41000366210938,223.25318908691406,30474000)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-12','MSFT',225.39999389648438,227.86000061035156,223.9600067138672,225.75,223.58995056152344,21903900)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-13','MSFT',219.85000610351562,236.10000610351562,219.1300048828125,234.24000549316406,231.99871826171875,42551800)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-14','MSFT',235.5399932861328,237.24000549316406,228.33999633789062,228.55999755859375,226.373046875,30198600)
%sql insert into public.stock_price(stock_date, stock_symbol, open_price, high_price, low_price, close_price, adjusted_close_price, volume)values('2022-10-17','MSFT',235.82000732421875,238.9600067138672,235.13999938964844,237.52999877929688,235.25721740722656,28142300)


In [ ]:
%sql select * from public.stock_price

## Part 3 - Query unstructured data in OpenSearch with vector search

In [ ]:
def semantic_search(query, k=5):
    print("semantic search input: " + query)
    search_vector = bedrock_embeddings.embed_query(query)
    query={
        "size": 5,
        "query": {
            "knn": {
                "item_vector":{
                    "vector":search_vector,
                    "k":5
                }
            }
        }
    }

    res = aos_client.search(index=index_name, 
                       body=query,
                       stored_fields=["company_name","item_category","item_content"])
    #print("Got %d Hits:" % res['hits']['total']['value'])
    query_result=[]
    for hit in res['hits']['hits']:
        row=[hit['fields']['item_content'][0]]
        query_result.append(row)

    query_result_df = pd.DataFrame(data=query_result,columns=["item_content"])
    return query_result_df

def semantic_search_full_field(query, k=5):
    search_vector = bedrock_embeddings.embed_query(query)
    query={
        "size": 5,
        "query": {
            "knn": {
                "item_vector":{
                    "vector":search_vector,
                    "k":5
                }
            }
        }
    }

    res = aos_client.search(index=index_name, 
                       body=query,
                       stored_fields=["company_name","item_category","item_content"])
    #print("Got %d Hits:" % res['hits']['total']['value'])
    query_result=[]
    for hit in res['hits']['hits']:
        row=[hit['_id'],hit['_score'],hit['fields']['company_name'][0],hit['fields']['item_category'][0],hit['fields']['item_content'][0]]
        query_result.append(row)

    query_result_df = pd.DataFrame(data=query_result,columns=["_id","_score","company_name","item_category","item_content"])
    return query_result_df


In [ ]:
query_result=semantic_search("what is Microsoft business?")

In [ ]:
query_result

## Part 4 - Query structred data  in Redshift with `SQLDatabaseChain`
A common use of an agent is to look up a record in a database. It would not be practical to include the full database in the context, so you can provide tools that perform actions against the datebase that eliminates hallucinations while maintining the conversational interactions.

### SQL Database Chain
Langchain has a SQL Database chain to ask questions of a DB to get answers. For details, read this document: https://python.langchain.com/docs/use_cases/qa_structured/sql#case-2-text-to-sql-query-and-execution


In [ ]:
from langchain.llms.bedrock import Bedrock
from typing import Optional, List, Any
from langchain.callbacks.manager import CallbackManagerForLLMRun

bedrock_llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock)
bedrock_llm.model_kwargs = {"max_tokens_to_sample":1204,"temperature":0.01,"top_k":250,"top_p":1,"stop_sequences":["\\n\\nHuman:"]}


In [ ]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

url = URL.create(
drivername='redshift+redshift_connector', # indicate redshift_connector driver and dialect will be used
host=redshift_serverless_endpoint, 
port=5439,
database='dev',
username=redshift_serverless_username,
password=redshift_serverless_password
)


db = SQLDatabase.from_uri(url,include_tables=['stock_symbol'])

from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Human: Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
If the results of the query is empty, answer \"I don't know\"
<format>
Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Answer with SQLResult. If SQLResult is empty, asnwer I don't know"
</format>
Assistant: Understood, I will use the above format and only provide the answer.

Only use the following tables:
<tables>
CREATE TABLE stock_symbol (
	stock_symbol text PRIMARY KEY,
	company_name text NOT NULL
)
</tables>

If someone asks for the table stock symbol table, they really mean the stock_symbol table.
<examples>
Question: What is the ticker symbol for Amazon in stock symbol table?
SQLQuery: SELECT stock_symbol FROM stock_symbol WHERE lower(company_name) ILIKE '%Amazon%'
SQLResult: AMZN
Answer: AMZN

Question: What is the ticker symbol for Microsoft in stock ticker table?
SQLQuery: SELECT stock_symbol FROM stock_symbol WHERE lower(company_name) ILIKE '%Microsoft%'
SQLResult: empty
Answer: I don't know

</examples>

Question: {input}

"""

PROMPT = PromptTemplate(
    input_variables=["input", "dialect"], template=_DEFAULT_TEMPLATE
)

llm=bedrock_llm

db_chain = SQLDatabaseChain.from_llm(
    llm, 
    db, 
    verbose=True, 
    return_intermediate_steps=True, 
    prompt=PROMPT, 
    )


In [ ]:
response = db_chain("\n\nHuman: What is the ticker symbol for Amazon in stock symbol table? \n\nAssistant:")
response['result']

In [ ]:
response = db_chain("\n\nHuman: What is the ticker symbol for MICROSOFT in stock ticker table? \n\nAssistant:")
response['result']

In [ ]:
response = db_chain("\n\nHuman: What is the ticker symbol for Pan Test in stock ticker table? \n\nAssistant:")
response['result']

In [ ]:
stock_price_template = """Human: Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
If the results of the query is empty, answer \"I don't know\"
<format>
Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Answer with SQLResult. If SQLResult is empty, asnwer I don't know"
</format>
Assistant: Understood, I will use the above format and only provide the answer.

Only use the following tables:
<tables>
CREATE TABLE public.stock_price (
stock_date DATE, 
stock_symbol text, 
open_price DECIMAL, 
high_price DECIMAL, 
low_price DECIMAL, 
close_price DECIMAL, 
adjusted_close_price DECIMAL, 
volume DECIMAL);
</tables>

answer the following question and organize the return data into json format.
1. what is the company average open price in the month of July? 
2. what is the company average close price in the month of July?
3. what is the company average high price in the month of July?
4. what is the company average low price in the month of July?
5. what is the company average adjusted close price in the month of July?
6. what is the company average volume in the month of July?

Question: {input}

"""

stock_prompt = PromptTemplate(
    input_variables=["input", "dialect"], template=stock_price_template
)

llm=bedrock_llm

stock_price_db_chain = SQLDatabaseChain.from_llm(
    llm, 
    db, 
    verbose=True, 
    return_intermediate_steps=True, 
    prompt=stock_prompt, 
    )


In [ ]:
stock_price_info = stock_price_db_chain("\n\nHuman: What is the MSFT stock information? \n\nAssistant:")

In [ ]:
print(stock_price_info['result'])

## Part 5 - Create LLM based autonomous agent for generative AI application

In [ ]:
import json
import time
import re
import requests

from langchain.agents import load_tools, AgentType, Tool, initialize_agent
from datetime import date
import pandas as pd
from datetime import datetime, timedelta

import warnings
import os
warnings.filterwarnings("ignore")
from langchain.tools import DuckDuckGoSearchRun
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain

def is_stock_related_query(query):
    template = """You are a helpful assistant to judge if the human input is stock related question.
    If it is stock related, answer \"yes\". Otherwise answer \"no\"."""
    human_template = "{text}"

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("human", human_template),
    ])

    llm_chain = LLMChain(
        llm=llm,
        prompt=chat_prompt
    )
    stock_related = llm_chain({"text":query})['text'].strip()
    return stock_related, query
    
def get_stock_ticker(query):
    template = """You are a helpful assistant who extract company name from the human input.Please only output the company"""
    human_template = "{text}"

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("human", human_template),
    ])

    llm_chain = LLMChain(
        llm=llm,
        prompt=chat_prompt
    )

    company_name=llm_chain(query)['text'].strip()
    company_ticker = db_chain("\n\nHuman: What is the ticker symbol for " + str(company_name) + " in stock ticker table? \n\nAssistant:")
    return company_name, company_ticker['result']

def get_stock_price(ticker):
    #get stock price with text to sql in db_chain
    data = stock_price_db_chain("\n\nHuman: What is the " + str(ticker) + " stock information ? \n\nAssistant:")
    return data['result']


# Get financial statements from OpenSearch
def get_financial_statements(query):
    company_statements = semantic_search(query)
    return company_statements

def anazlyze_stock(query):
    company_name,ticker=get_stock_ticker(query)
    print({"Query":query,"company_name":company_name,"Ticker":ticker})
    stock_data=get_stock_price(ticker)
    financial_statements=get_financial_statements(query)

    available_information=f"Stock Price: {stock_data}\n\nStock Financials: {financial_statements}"
    analysis=llm(f"""\n\nHuman: Give detail stock analysis, Use the available data and provide investment recommendation. \
             The user is fully aware about the investment risk, dont include any kind of warning like 'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer \
             User question: {query} \
             You have the following information available about {company_name}. Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explaination.Try to Give positives and negatives  : \
              {available_information} \
                \n\nAssistant: """
             )
    return analysis

In [ ]:
is_stock_related_query("What is SageMaker?")

In [ ]:
is_stock_related_query("What is the main business of Microsoft?")

In [ ]:
company_name, company_ticker = get_stock_ticker("What is the main business of Microsoft?")
print(company_name)
print(company_ticker)

In [ ]:
print(get_stock_price("MSFT"))

In [ ]:
print(get_financial_statements("MSFT"))

In [ ]:
analyze_result=anazlyze_stock("Is Microsoft a good investment choice right now?")
print(analyze_result)

## Part 6 - Using ReAct Agent in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain

tools=[
    Tool(
        name="is stock related query",
        func=is_stock_related_query,
        description="If the query is stock related"
    ),
    Tool(
        name="get company ticker",
        func=get_stock_ticker,
        description="Get the company stock ticker"
    ),
    Tool(
        name="get stock data",
        func=get_stock_price,
        description="Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it "
    ),
    Tool(
        name="get financial statements",
        func=get_financial_statements,
        description="Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it"
    ) 
]


In [ ]:
updated_prompt="""Human: You are a financial advisor. Give stock recommendations for given query based on following instructions. 
<instructions>
Answer the following questions as best you can. You have access to the following tools:

is stock related query: Use when you need to know whether this is stock related query. This tool will output whether human input is stock related and human input. You should input the human input to it.
get company ticker: Use when you need to extract company name and stock ticker. This tool will output company name and stock ticker.
get stock data: Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the stock ticker to it.
get financial statements: Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it
</instructions>

<steps>
Note- if you fail in satisfying any of the step below, Just move to next one
1) Use "is stock related query" tool to judge if the input query is stock related or not. Output - stock related and input query
2) Use "get company ticker" tool to get the company name and stock ticker. Output- company name and stock ticker
3) Use "get stock data" tool to gather stock info. Output- Stock data
4) Use "get financial statements" tool to get company's historic financial data. Output- Financial statement
5) Analyze the stock based on gathered data and give detail analysis for investment choice. provide numbers and reasons to justify your answer. Output- Detailed stock Analysis
</steps>

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [is stock related query, get company ticker, get stock data, get financial statements]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}

"""

In [ ]:
from langchain.agents import initialize_agent 

zero_shot_agent=initialize_agent(
    llm=llm,
    agent="zero-shot-react-description",
    tools=tools,
    verbose=True,
    max_iteration=2,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
)

zero_shot_agent.agent.llm_chain.prompt.template=updated_prompt

In [ ]:
response = zero_shot_agent("\n\nHuman: Is Microsoft a good investment choice right now? \n\nAssistant:")

In [ ]:
print(response["output"])

In [ ]:
print(response["intermediate_steps"])

In [ ]:
response = zero_shot_agent("\n\nHuman: What is SageMaker? \n\nAssistant:")

In [ ]:
print(response)

## Restore Configuration

In [ ]:
get_response = redshift_serverless_client.get_workgroup(
    workgroupName=redshift_serverless_workgroup
)

print("publiclyAccessible: " + str(get_response['workgroup']['publiclyAccessible']))

if get_response['workgroup']['publiclyAccessible'] == True:
    redshift_serverless_client.update_workgroup(workgroupName=redshift_serverless_workgroup, publiclyAccessible=False)

get_response = redshift_serverless_client.get_workgroup(
    workgroupName=redshift_serverless_workgroup
)
update_status = get_response['workgroup']['status']

while update_status == 'MODIFYING':
    time.sleep(2)
    print('.', end='')
    get_response = redshift_serverless_client.get_workgroup(
    workgroupName=redshift_serverless_workgroup
    )
    update_status = get_response['workgroup']['status']
    
get_response = redshift_serverless_client.get_workgroup(
    workgroupName=redshift_serverless_workgroup
)

print("publiclyAccessible: " + str(get_response['workgroup']['publiclyAccessible']))
